# Conversion for a specific Excel template

Takes an Excel file written in a specific way and converts it into a generic shape import csv file.

## Copyright information

Copyright © 2015 Simon Biggs

This program is free software: you can redistribute it and/or modify it under the terms of the GNU Affero General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU Affero General Public License for more details.

You should have received a copy of the GNU Affero General Public License along with this program. If not, see http://www.gnu.org/licenses/.

In [1]:
import pandas as pd
import numpy as np

In [2]:
filepath = '../../../user_input/DataImport/GenericShapeImport/Electron_Output_Factors_LA4_v1.xlsx'
rawData = pd.read_excel(filepath,skiprows=32,sheetname=1)

In [3]:
rawOutputFactor = np.zeros(np.shape(rawData['Output Factor'].values))

for i, loadedFactor in enumerate(rawData['Output Factor'].values):
    
    try:
        rawOutputFactor[i] = np.float(loadedFactor)
        
    except:
        rawOutputFactor[i] = np.nan

In [4]:
useData = ((rawData['Use Factor (Y/N)'].values != '-') & 
 (rawData['Use Factor (Y/N)'].values != 'N') &
 (~np.isnan(rawOutputFactor)))

In [5]:
factor = rawOutputFactor[useData]

In [6]:
shape = [0,]*sum(useData)

for i, loadedShape in enumerate(rawData['Shape'].values[useData]):
    shape[i] = loadedShape.lower()

In [7]:
width = np.zeros(sum(useData))
length = np.zeros(sum(useData))

for i, insertString in enumerate(rawData['Insert (cm x cm)'][useData]):
    
    dimenstionsString = insertString.lower().split("x")
    
    sideA = float(dimenstionsString[0])
    sideB = float(dimenstionsString[1])
    
    if sideA < sideB:
        width[i] = sideA
        length[i] = sideB
        
    else:
        width[i] = sideB
        length[i] = sideA

In [8]:
applicator = np.zeros(sum(useData))

for i, applicatorString in enumerate(rawData['Applicator (cm x cm)'].values[useData]):
    
    applicator[i] = float(applicatorString.lower().split("x")[0])

In [9]:
energy = np.zeros(sum(useData))

for i, energyString in enumerate(rawData['Energy (MeV)'].values[useData]):
    
    energy[i] = float(energyString)

In [10]:
ssd = np.zeros(sum(useData))

for i, ssdString in enumerate(rawData['SSD (cm)'].values[useData]):
    
    ssd[i] = float(ssdString)

In [11]:
# Can import a bulk label here which can be attached to each shape in the model

In [23]:
factorIndex = [0,] * sum(useData)

for i, index in enumerate(factorIndex):
    
    factorIndex[i] = "data collection " + str(i)

In [24]:
index = ['index','energy', 'applicator', 'width', 'length', 'shape', 'ssd', 'factor']
data2export = np.vstack([factorIndex,energy,applicator,width,length,shape,ssd,factor])

exportDataFrame = pd.DataFrame(data2export, index=index).T

exportDataFrame.to_csv('../../../user_input/DataImport/GenericShapeImport/GenericShapeImport.csv',index=None)